In [6]:
import pandas as pd
import json
import requests
import os


In [7]:
with open('/root/code/juancruzgui/Projects/meli_houses/full_neigh_1.json', 'r') as file:
    neigh_dict_lists = json.load(file)


In [8]:
df = pd.DataFrame(neigh_dict_lists)


In [9]:
df['city_name'] = df.city.apply(lambda x: x['name'])
df['city_id'] = df.city.apply(lambda x: x['id'])
df['state_name'] = df.state.apply(lambda x: x['name'])
df['state_id'] = df.state.apply(lambda x: x['id'])
df['country_name'] = df.country.apply(lambda x: x['name'])
df = df.dropna(subset = 'geo_information')
df['latitude'] = df.geo_information.apply(lambda x: x.get('location', None).get('latitude', None) if x.get('location', None) is not None else None)
df['longitude'] = df.geo_information.apply(lambda x: x.get('location', None).get('longitude', None) if x.get('location', None) is not None else None)
df = df.drop(columns = ['city', 'state', 'country', 'geo_information', 'subneighborhoods'])
df = df.dropna(subset=['latitude', 'longitude'])
df['latitude_r1'] = df['latitude'].apply(lambda x: x - 0.04)
df['latitude_r2'] = df['latitude'].apply(lambda x: x + 0.04)
df['longitude_r1'] = df['longitude'].apply(lambda x: x - 0.04)
df['longitude_r2'] = df['longitude'].apply(lambda x: x + 0.04);


In [10]:
df.to_csv('neighborhoods.csv')


In [11]:
df#[df.state_name == 'Mendoza'].sort_values(by = 'latitude').head(20)


,id,name,city_name,city_id,state_name,state_id,country_name,latitude,longitude,latitude_r1,latitude_r2,longitude_r1,longitude_r2
0,TVhYQXJlbmFzIDE5VFV4QlEwTlBVM1JoWlhOd,Arenas 19,Costa Esmeralda,TUxBQ0NPU3RhZXNt,Bs.As. Costa Atlántica,TUxBUENPU2ExMmFkMw,Argentina,-37.005848,-56.795856,-37.045848,-36.965848,-56.835856,-56.755856
1,TVhYQm9zcXVlVFV4QlEwTlBVM1JoWlhOdA,Bosque,Costa Esmeralda,TUxBQ0NPU3RhZXNt,Bs.As. Costa Atlántica,TUxBUENPU2ExMmFkMw,Argentina,-37.002484,-56.804925,-37.042484,-36.962484,-56.844925,-56.764925
2,TVhYRGVwb3J0aXZvIDFUVXhCUTBOUFUzUmhaW,Deportivo 1,Costa Esmeralda,TUxBQ0NPU3RhZXNt,Bs.As. Costa Atlántica,TUxBUENPU2ExMmFkMw,Argentina,-37.009678,-56.802687,-37.049678,-36.969678,-56.842687,-56.762687
3,TVhYRGVwb3J0aXZvIDJUVXhCUTBOUFUzUmhaW,Deportivo 2,Costa Esmeralda,TUxBQ0NPU3RhZXNt,Bs.As. Costa Atlántica,TUxBUENPU2ExMmFkMw,Argentina,-37.010671,-56.795220,-37.050671,-36.970671,-56.835220,-56.755220
4,TVhYRWN1ZXN0cmVUVXhCUTBOUFUzUmhaWE50,Ecuestre,Costa Esmeralda,TUxBQ0NPU3RhZXNt,Bs.As. Costa Atlántica,TUxBUENPU2ExMmFkMw,Argentina,-37.022634,-56.819143,-37.062634,-36.982634,-56.859143,-56.779143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5839,TUxBQlNBTjk5NDda,San Pedro de Colalao,Trancas,TUxBQ1RSQTgyMTM,Tucumán,TUxBUFRVQ244NmM3,Argentina,-26.233330,-65.483330,-26.273330,-26.193330,-65.523330,-65.443330
5840,TUxBQlZJTDIzNjVa,Villa de Trancas,Trancas,TUxBQ1RSQTgyMTM,Tucumán,TUxBUFRVQ244NmM3,Argentina,-26.216129,-65.283508,-26.256129,-26.176129,-65.323508,-65.243508
5841,TUxBQkJBUjkxNzVa,Barrio San José III,Yerba Buena,TUxBQ1lFUjgyOWU0,Tucumán,TUxBUFRVQ244NmM3,Argentina,-26.783010,-65.239624,-26.823010,-26.743010,-65.279624,-65.199624
5842,TUxBQlZJTDY4NjFa,Villa Carmela,Yerba Buena,TUxBQ1lFUjgyOWU0,Tucumán,TUxBUFRVQ244NmM3,Argentina,-26.750000,-65.283340,-26.790000,-26.710000,-65.323341,-65.243340


In [19]:
#Total 5843 rows
#Chunks of 100 houses
def get_houses_links(df):
    houses = 0
    for i in range(0,len(df),100):
        houses_list = []
        print(f'\nTotal houses retrieved: {houses}')
        print(f"\nWorking on range {i}-{i+100}")
        for j in range(100):
            if not os.path.exists(f'houses2/houses_links_{i}-{i+99}.json'):
                print(f'\nworking on {i+j}/{i+100} (5843 Total)')
                state_id = df.loc[j,'state_id']
                city_id = df.loc[j,'city_id']
                neigh_id = df.loc[j,'city_id']

                page = 0
                #https://api.mercadolibre.com/sites/MLA/search?&category=MLA1468&state={state_id}&city={city_id}&neighborhood={neigh_id}
                response = requests.get(f'https://api.mercadolibre.com/sites/MLA/search?&category=MLA1468&state={state_id}&city={city_id}&neighborhood={neigh_id}')
                response_dict = json.loads(response.text)

                total = response_dict['paging']['primary_results']
                results = response_dict['results']
                for result in results:
                    houses_list+=result['id']
                    with open('ids.txt', 'a') as file:
                        file.write(f'{result["id"]}\n')

                houses+=total
                print(f"Total houses results: {total}")

                if total>50:
                    for offset in range(50,total,50):
                        print(f"Offset {offset}/{total}")
                        response = requests.get(f'https://api.mercadolibre.com/sites/MLA/search?&category=MLA1468&state={state_id}&city={city_id}&neighborhood={neigh_id}&offset={offset}')
                        response_dict = json.loads(response.text)

                        results = response_dict['results']
                        for result in results:
                            houses_list+=result['id']
                            with open('App/ML_API/ids/ids.txt', 'a') as file:
                                file.write(f'{result["id"]}\n')

            else:
                print('Range already retrieved')
                break
        if not os.path.exists(f'houses2/houses_{i}-{i+99}.json'):
            with open(f'houses2/houses_{i}-{i+j}.json','w') as file:
                file.write(str(houses_list))
        else:
            continue
    return None
